In [1]:
from meta_strategy import meta_strategy_prompt
from meta_think import meta_think_prompt
from struct_think import struct_think

In [2]:
print(meta_think_prompt+meta_strategy_prompt+struct_think)


你是一名元认知科学家，按照如下的结构进行元认知分析：
<struct>
    1. 元认知知识: 你对自己和他人在思考、解决问题和学习过程方面的了解。包括陈述性知识，程序性知识、条件性知识。
        1. 陈述性知识：关于自己和任务的事实，不包含任务本身的事实
        2. 程序性知识：知道如何执行策略
        3. 条件性知识：知道何时及为何使用策略
    2. 元认知调节: 你用来控制思维的活动和策略。包括规划、监控和评估。
        1. 规划：在开始任务前制定目标和选择策略。例如，决定在阅读前先浏览标题和问题以了解重点。
        2. 监控：在任务进行中检查进展和理解程度。例如，边读边问自己“我明白了吗？” 
        3. 评估：任务完成后反思结果和策略的有效性。例如，考试后回顾哪些复习方法最有用。
    3. 元认知体验: 学习新知识或尝试解决问题时产生的想法和感受。以下是其主要类型：
        1. 知道感（Feeling of Knowing, FOK）  
            定义：在回忆信息前，觉得自己知道答案的一种感觉。  
            示例：考试时，看到问题后觉得答案就在嘴边，但尚未完全回忆起来。  
            作用：这种感觉可以指导是否继续努力回忆或转向其他策略。
        2. 舌尖现象（Tip-of-the-Tongue Experience, TOT）  
            定义：知道某事但暂时无法回忆起来的感觉，通常伴随部分相关信息的激活。  
            示例：试图回忆演员的名字，但只记得他们的电影角色。  
            作用：这种体验提示记忆检索的接近性，可能促使个体通过提示或上下文辅助回忆。
        3. 信心判断（Confidence Judgments）  
            定义：对自己答案或决定的确定程度评估。  
            示例：在选择题中，感觉某个选项非常正确，决定不更改。  
            作用：影响决策，如是否坚持答案或重新检查。
        4. 难度评估（Difficulty Assessments）  
            定义：感知任务

In [ ]:
import os
import ollama
from pydantic import BaseModel
import json

class MetaKnolege(BaseModel):
    Declarative_Knowledge: list[str]
    Procedural_Knowledge: list[str]
    Conditional_Knowledge: list[str]
    
class MetaRegulation(BaseModel):
    Planning: list[str]
    Monitoring: list[str]
    Evaluating: list[str]
    
class MetaExperience(BaseModel):
    Positive: list[str]
    Negative: list[str]
    Neutral: list[str]
    
class MetaStrategy(BaseModel):
    SelfQuestion: str
    Reflection: str
    GoalSetting: str
    Modeling: str
    Scaffolding: str

class MetaThink(BaseModel):
    MetaStrategy: MetaStrategy
    MetaKnolege: MetaKnolege
    MetaRegulation: MetaRegulation
    MetaExperience: MetaExperience

class StructedThink(BaseModel):
    DecomposedQuestion: list[str]
    Order: list[int]
    Tools: list[str]
    Skills: list[str]
    
class InvestmentDecision(BaseModel):
    Time: str
    Investor: str
    Amount: str
    Object: str
    
class InvestProblemSolve(BaseModel):
    MetaThink: MetaThink
    StructedThink: StructedThink
    Steps: str
    InvestmentDecision: InvestmentDecision

class InvestmentBackground(BaseModel):
    ObjectToBeInvested: str
    Time: str
    BackgroundInfo: str


    
class InvestProblem(BaseModel):
    MetaThink: MetaThink
    InvestmentBackground: InvestmentBackground
    InvestmentDecision: InvestmentDecision
    InvestmentReturn: str
    
def process_text_with_prompt(system_prompt, user_prompt):
    """使用ollama处理文本，支持system prompt和用户prompt的组合"""
    # 构造ollama需要的消息格式
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    # 调用ollama的chat接口
    response = ollama.chat(
        model='qwq',  # 根据实际模型调整，如'mistral'
        messages=messages,
        format=InvestProblemSolve.model_json_schema(),
        options={"num_ctx":7000}
    )
    return response['message']['content']

def process_file_with_ollama(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        
        # 定义提示词
        system_prompt = meta_think_prompt+meta_strategy_prompt
        user_prompt = f"""
        从下面案例中抽取出
        投资问题描述：
            1. 元认知：元认知知识、元认知调节、元认知体验、元认知策略。
            1. 结构化思考：分解后的问题，问题对应的解决顺序，使用的工具，应用的技巧
            1. 思考的过程：一步一步按照结构化思维进行推理的过程
            2. 最终投资决策：不要超过150字，包括时间、投资人、投资金额，投资对象。
        <case>
            {content}
        </case>
        """
        #print(f"user_prompt: {user_prompt}")
        # 调用处理函数
        result = process_text_with_prompt(
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        result = json.loads(result)
        result["think"] = content
        return result

# 遍历output目录里面的每一个文件
output_dir = "knowledge/output"
results = []
for filename in os.listdir(output_dir):
    file_path = os.path.join(output_dir, filename)
    if os.path.isfile(file_path):
        result = process_file_with_ollama(file_path)
        print(f"Processed {filename}: {result}")
        results.append(result)
    break

# 将结果写入文件
output_file = "corpus/output.json"
with open(output_file, "w+", encoding='utf-8') as f:
    for result in results:
        # format json string to one line
        result = json.dumps(result, ensure_ascii=False)
        f.write(f"{result}\n")

TypeError: 'str' object does not support item assignment

In [10]:
print(json.loads(result))

{"MetaThink": {"MetaStrategy": {"SelfQuestion": "在收购国民保险公司时，巴菲特考虑了哪些关键因素？如何评估公司的真实价值？浮存金的作用是什么？", "Reflection": "收购后通过优化投资组合使资产增值，验证了浮存金策略的有效性。需持续关注保险业务的承保质量和浮存金增长潜力。", "GoalSetting": "目标是将伯克希尔现金流投入高收益稳健企业，要求收购价格显著低于内在价值，确保年化收益率超过20%。", "Modeling": "模仿格雷厄姆的价值投资方法，寻找被低估且有安全边际的公司。", "Scaffolding": "初期借助瑞沃茨管理团队经验，后续逐步介入浮存金投资策略优化。"}, "MetaKnolege": {"Declarative_Knowledge": ["国民保险公司的业务范围和经营哲学","保险浮存金的操作原理","巴菲特的价值投资理念"], "Procedural_Knowledge": ["分析财务数据（已赚保费、净利润）","评估市盈率合理性","测算浮存金可投资规模"], "Conditional_Knowledge": ["当发现承保利润稳定时采用低估值收购策略","当获得保险浮存金时优先用于复利投资"]}, "MetaRegulation": {"Planning": ["分析瑞沃茨的经营状况和出售意愿","计算公司内在价值与报价差异","评估浮存金潜力"], "Monitoring": ["跟踪收购后的财务表现","观察承保利润波动情况","监控浮存金投资回报"], "Evaluating": ["验证收购价格是否安全边际充足","确认浮存金策略有效性","总结保险业务整合经验"]}, "MetaExperience": {"Positive": ["对瑞沃茨管理能力的信心增强购买决心","浮存金规模超预期带来惊喜感"],"Negative":["对承保风险波动的担忧影响报价策略","短期现金流压力下的决策焦虑"], "Neutral": ["客观评估财务数据支撑决策","冷静应对市场噪音干扰"]}}, "InvestmentBackground": {"ObjectToBeInvested": "国民保险公司（National Indemnity Company）